### Mercado Bitcoin

Será utilizado como exemplo a API da plataforma mercado bitcoin

url: https://api.mercadobitcoin.net/api/v4/docs


In [ ]:
import os
import logging
import requests
from abc import ABC, abstractmethod
import datetime
from typing import List
import json


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [ ]:
class DataTypeNotSupportedException(Exception):
    def __init__(self, data) -> None:
        self.data = data
        self.message = f'Data type {type(data)} is not supported for ingestion'
        super().__init__(self.message)

In [ ]:
class DataWriter():
    def __init__(self, coin: str, api: str) -> None:
        self.coin = coin
        self.api = api
        self.filename = f'{self.api}/{self.coin}/{datetime.datetime.now()}.json'

    def _write_row(self, row: str) -> None:
        os.makedirs(os.path.dirname(self.filename), exist_ok=True)
        with open(self.filename, 'a') as f:
            f.write(row)

    def write(self, data):
        if isinstance(data, dict):
            self._write_row(json.dumps(data) + '\n')
        elif isinstance(data, List):
            for element in data:
                self.write(element)
        else:
            raise DataTypeNotSupportedException(data)

In [ ]:
class MercadoBitcoinApi(ABC):
    def __init__(self, coin: str):
        self.coin = coin
        self.base = 'https://api.mercadobitcoin.net/api/v4'

    @abstractmethod
    def _get_endpoint(self) -> str:
        pass

    def get_data(self, **kwargs) -> dict:
        endpoint = self._get_endpoint(**kwargs)
        logger.info(f'Getting data from endpoint: {endpoint}')
        response = requests.get(endpoint)
        return response.json()


class TickersApi(MercadoBitcoinApi):
    type = 'tickers'

    def _get_endpoint(self) -> str:
        return f'{self.base}/{self.type}/?symbols={self.coin}'


class TradesApi(MercadoBitcoinApi):
    type = 'trades'

    def _get_unix_epoch(self, date: datetime) -> int:
        return int(date.timestamp())

    def _get_endpoint(self, date_from: datetime.datetime = None, date_to: datetime.date = None) -> str:

        if date_from and not date_to:
            unix_date_from = self._get_unix_epoch(date_from)
            endpoint = f'{self.base}/{self.coin}/{self.type}/?from={unix_date_from}'
        elif date_from and date_to:
            unix_date_from = self._get_unix_epoch(date_from)
            unix_date_to = self._get_unix_epoch(date_to)
            endpoint = f'{self.base}/{self.coin}/{self.type}/?from={unix_date_from},to={unix_date_to}'
        else:
            endpoint = f'{self.base}/{self.coin}/{self.type}'

        return endpoint

In [ ]:
class DataIngestor(ABC):
    def __init__(self, writer: DataWriter, coins: List[str], default_start_date: datetime.datetime = None) -> None:
        self.writer = writer
        self.coins = coins
        self.default_start_date = default_start_date

    @abstractmethod
    def ingest(self) -> None:
        pass

class TickersIngestor(DataIngestor):
    def ingest(self) -> None:
        for coin in self.coins:
            api = TickersApi(coin)
            data = api.get_data()
            self.writer(coin=coin, api=api.type).write(data)

class TradesIngestor(DataIngestor):
    def ingest(self) -> None:
        date = self.default_start_date
        if date < datetime.datetime.today():
            for coin in self.coins:
                api = TradesApi(coin)
                data = api.get_data()
                self.writer(coin=coin, api=api.type).write(data)

In [ ]:
ingestorTickers = TickersIngestor(writer=DataWriter, coins=['BTC-BRL', 'ETH-BRL'])
ingestorTickers.ingest()

ingestorTrades = TradesIngestor(writer=DataWriter, coins=['BTC-BRL', 'ETH-BRL'], default_start_date=datetime.datetime(2022,7,30))
ingestorTrades.ingest()